In [ ]:
# default_exp experimental

In [ ]:
#hide
from IPython.display import clear_output
from nbdev.export import notebook2script
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import json
import logging

logger = logging.getLogger()
logger.setLevel("INFO")

# experimental

> description

In [ ]:
def str2tensor(s):
    lines = s.split("\n")
    tuples = [l.split("\t") for l in lines]
    tuples = [[int(e) for e in t] for t in tuples]
    return torch.tensor(tuples, dtype=torch.float32)

## Trainer

In [ ]:
#server
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# trainer = Trainer(%)
ModelCheckpoint()

## Ideas

- model architecture
    - dcgan
    - resnet
- SAGAN:
    - Spectural norm
    - WGAN-GP
    - TTUR: 調整 D 以及 G 的 LR Or D 多訓練幾次
- 使用WGAN 系列、SNGAN、加 dropout layer、對 input image 加上 noise。
- upsample+conv2d 组合代替 transposed_conv2d，可以减少 checkerboard 的产生


### Weight init

```python
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
```

### Optim: Adam or RMSprop

```python
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)
```

### Discriminator Weight clippling

```python
parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")


for p in discriminator.parameters():
    p.data.clamp_(-opt.clip_value, opt.clip_value)
```

## 訓練指令

### MNIST

```bash
#gan
python applications/image_synthesis/train_gan.py\
    --dataset mnist\
    --latent_dim 128\
    --dim 32\
    --channels 1\
    --batch_size 256
   
# wgan
python applications/image_synthesis/train_gan.py\
    --dataset mnist\
    --adversarial_loss_type wgan\
    --latent_dim 128\
    --dim 32\
    --channels 1\
    --batch_size 256
    
# wgan + weight clip
python applications/image_synthesis/train_gan.py\
    --dataset mnist\
    --adversarial_loss_type wgan\
    --discriminator_weight_clip_value 0.01\
    --latent_dim 128\
    --dim 32\
    --channels 1\
    --batch_size 256
    
# sngan: spectral normalization in d
python applications/image_synthesis/train_gan.py\
    --dataset mnist\
    --adversarial_loss_type wgan\
    --discriminator_spectral_norm True\
    --latent_dim 128\
    --dim 32\
    --channels 1\
    --batch_size 256
    
# hinge
python applications/image_synthesis/train_gan.py\
    --dataset mnist\
    --adversarial_loss_type wgan\
    --discriminator_spectral_norm True\
    --latent_dim 128\
    --dim 32\
    --channels 1\
    --batch_size 256
```

### Crykpo

```bash
python applications/image_synthesis/train_gan.py\
    --dataset crypko\
    --adversarial_loss_type wgan\
    --discriminator_weight_clip_value 0.01\
    --latent_dim 128\
    --dim 64\
    --channels 3\
    --batch_size 512
    
# sngan
python applications/image_synthesis/train_gan.py\
    --dataset crypko\
    --adversarial_loss_type hinge_v1\
    --discriminator_spectral_norm True\
    --latent_dim 128\
    --dim 64\
    --channels 3\
    --batch_size 512
```

### Stickers

```python
# wgan + weight clip
python applications/image_synthesis/train_gan.py\
    --dataset stickers\
    --adversarial_loss_type wgan\
    --discriminator_weight_clip_value 0.01\
    --latent_dim 128\
    --dim 64\
    --channels 3\
    --batch_size 1024
```

Train

```bash
python applications/gan/train.py stickers 128 128 3 \
    --batch_size 256 \
    --max_epochs 1


python applications/gan/train.py stickers 128 128 3 lsgan \
    --batch_size 512 \
    --max_epochs 100
    
    
python applications/image_generation/train_gan.py crypko 64 64 3 lsgan \
    --batch_size 512 \
    --max_epochs 10000
    
```

Clean up

```bash
rm -r checkpoints/
rm -r lightning_logs/
```

In [ ]:
#server
# for _ in range(100):
#     python examples/gan/train.py stickers 128 128 3 \
#         --batch_size 256 \
#         --max_epochs 100

## Inference

In [ ]:
#server
# import torch
# from practical_ai.vision.gan.core import GAN
# model = GAN.load_from_checkpoint("../models/_ckpt_epoch_0.ckpt")
# model = model.eval()

# z = torch.randn(1, 128)
# img = model(z)

In [ ]:
#server
# !rm -rf ../models/_ckpt_epoch*